In [ ]:
!pip install recordlinkage

     |████████████████████████████████| 952kB 5.3MB/s 
     |████████████████████████████████| 92kB 7.4MB/s 


In [ ]:
import re
import recordlinkage
import numpy as np
from google.colab import drive
import os
import glob
import pandas as pd


# DATA PREFIX

In [ ]:
drive.mount('/content/mydrive')


Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
# Read Data in multi folder
folder_path = r'/content/mydrive/MyDrive/Data2'
subfolder_path = glob.glob(folder_path + "/*")
seq = []
for paths in subfolder_path:
  filenames = glob.glob(paths + "/*.csv")
  dfs = []
  for filename in filenames:
      dfs.append(pd.read_csv(filename))
  # Concatenate all data into one DataFrame
  seq.append(pd.concat(dfs, ignore_index=True))

data_df = pd.concat(seq, ignore_index=True)
data_df.shape

(3359, 24)

In [ ]:
list_idx = []
for i in range(0,len(data_df)):
  if data_df['pin'][i] == 'pin':
    list_idx.append(i)
print(list_idx)
data_df = data_df.drop(list_idx,axis=0)
data_df.reset_index(drop=True, inplace=True)

[1935, 1998, 2065, 2122, 2213, 2257, 2274, 2371, 2452, 2551, 2570, 2618, 2640, 2685, 2700, 3076, 3205]


In [ ]:
def isNaN(string):
    return string != string
list_idx = []
for i in range(0,len(data_df['name'])):
    if isNaN(data_df['name'][i]):
        list_idx.append(i)
data_df.drop(list_idx,axis=0,inplace=True)
data_df.reset_index(drop=True, inplace=True)
data_df.shape

(3342, 24)

In [ ]:
# get status of product from name

for i in range(0,len(data_df['name'])):
  try:
    b = re.findall('99%',data_df['name'][i])
    c = re.findall('97%',data_df['name'][i])
    a = (re.findall('cũ|98%|96%|95%|90%|likenew|Cũ|Likenew|LIKENEW',data_df['name'][i]))
    if b != []:
      data_df['status'][i]='99%'
    elif c != []:
      data_df['status'][i]='97%'
    elif a == []:
      data_df['status'][i]='mới'
    else:
      data_df['status'][i]='cũ'
  except:
    continue

In [ ]:
# explode exist color
data_df['color'] = data_df['color'].str.split(",")
data_df = data_df.explode('color',ignore_index=True)


In [ ]:
# get color of product from name

for i in range(0,len(data_df['name'])):
  if isNaN(data_df['color'][i]):
    try:
      a = re.findall('(bạc|tím|trắng|vàng|xanh|đỏ|lục|lam|đen|xám|gold|graphite|silver|blue|gray)',data_df['name'][i].lower())
      if a:
        data_df['color'][i] = ','.join(a)
    except:
      continue

In [ ]:
# explode exist color lan 2
data_df['color'] = data_df['color'].str.split(",")
data_df = data_df.explode('color',ignore_index=True)


In [ ]:
# regex to remove special string color
for i in range(0,len(data_df['color'])):
  try:
    data_df['color'][i] = re.sub('(nhôm nguyên khối|vỏ nguyên khối)','',data_df['color'][i].lower())
    data_df['color'][i] = re.sub('\(.+','',data_df['color'][i].lower())
  except:
    continue

In [ ]:
# explode color lan chot
data_df['color'] = data_df['color'].str.split("-")
data_df = data_df.explode('color',ignore_index=True)

data_df['color'] = data_df['color'].str.split("/")
data_df = data_df.explode('color',ignore_index=True)

for i in range(0,len(data_df['color'])):
  try:
    data_df['color'][i] = re.sub('^ +','',data_df['color'][i].lower())
    data_df['color'][i] = re.sub(' +$','',data_df['color'][i].lower())
    data_df['color'][i] = re.sub(' +',' ',data_df['color'][i].lower())
  except:
    continue

In [ ]:
for i in range(0,len(data_df['color'])):
  try:
    if data_df['color'][i]=='black':
      data_df['color'][i] = 'đen'
    elif data_df['color'][i]=='blue':
      data_df['color'][i] = 'xanh'
    elif data_df['color'][i]=='gold':
      data_df['color'][i] = 'vàng'
    elif re.findall('(grey|gray)',data_df['color'][i]):
      data_df['color'][i] = 'xám'
    elif data_df['color'][i]=='silver':
      data_df['color'][i] = 'bạc'
    elif re.findall('(rose)',data_df['color'][i]):
      data_df['color'][i] = 'hồng'
    elif re.findall('(graphite)',data_df['color'][i]):
      data_df['color'][i] = 'vàng đồng'
    elif re.findall('(xanh lá cây)',data_df['color'][i]):
      data_df['color'][i] = 'xanh lục'
    elif re.findall('(xanh dương)',data_df['color'][i]):
      data_df['color'][i] = 'xanh'
    elif re.findall('(bạc trắng)',data_df['color'][i]):
      data_df['color'][i] = 'bạc'
  except:
    continue

In [ ]:
for i in range(0,len(data_df['color'])):
  if data_df['color'][i]=='undefined':
      data_df['color'][i] = 'chưa rõ'

In [ ]:
data_df['color'] = data_df['color'].fillna(value='chưa rõ')

In [ ]:
data_df.shape

(4007, 24)

In [ ]:
# regex form of ram and explode ram

for i in range(0,len(data_df['ram'])):
  try:
    a = (re.findall('(\d+)(?=(?:\/\d+)*\s*([MG]B))',data_df['ram'][i]))
    for j in range(0,len(a)):
      a[j] = ''.join(a[j])
    data_df['ram'][i]=a
  except:
    continue
data_df = data_df.explode('ram',ignore_index=True)

In [ ]:
# regex form of rom and explode rom

for i in range(0,len(data_df['rom'])):
  try:
    a = (re.findall('(\d+)(?=(?:\/\d+)*\s*([MG]B))',data_df['rom'][i]))
    for j in range(0,len(a)):
      a[j] = ''.join(a[j])
    data_df['rom'][i]=a
  except:
    continue
data_df = data_df.explode('rom',ignore_index=True)

In [ ]:
# regex rear cam and front cam. after regex, things will only contain MP

for i in range(0,len(data_df['rear_cam'])):
  try:
    a = re.findall('\d+\.?\d+ MP',data_df['rear_cam'][i])
    data_df['rear_cam'][i] = '+'.join(a)
  except:
    continue
for i in range(0,len(data_df['front_cam'])):
  try:
    a = re.findall('\d+\.?\d+ MP',data_df['front_cam'][i])
    data_df['front_cam'][i] = '+'.join(a)
  except:
    continue

In [ ]:
# get status of product. now have 2 status: cu, moi

for i in range(0,len(data_df['name'])):
  a = (re.findall('cũ|99%|98%|97%|96%|95%|90%|likenew|Cũ|Likenew|LIKENEW',data_df['name'][i]))
  if a == []:
    data_df['status'][i]='mới'
  else:
    data_df['status'][i]='cũ'

In [ ]:
# pricing regex

for i in range(0,len(data_df['price'])):
  if not isNaN(data_df['price'][i]):
    a = (re.findall('[0-9]',data_df['price'][i]))
    data_df['price'][i] = ''.join(a)

In [ ]:
# sim regex

for i in range(0,len(data_df['name'])):
  a = (re.findall('wifi|WIFI|Wifi',data_df['name'][i]))
  b = (re.findall('4G|3G',data_df['name'][i]))
  if a and b:
    data_df['sim'][i] = 'e-sim'
  elif a and not b:
    data_df['sim'][i] = 'wifi only'

In [ ]:
for i in range(0,len(data_df['name'])):
  try:
    ip = re.findall('(iphone [0-9][0-9]|iphone [0-9]s|iphone se|iphone xr|iphone xs|iphone x|iphone [0-9])',data_df['name'][i].lower())
    pro = re.findall('(pro)',data_df['name'][i].lower())
    max = re.findall('(max)',data_df['name'][i].lower())
    plus = re.findall('(plus)',data_df['name'][i].lower())
    year = re.findall('(\([0-9][0-9][0-9][0-9]\)| [0-9][0-9][0-9][0-9] )',data_df['name'][i].lower())
    if ip:
      new_name = ''.join(ip) +' '+ ''.join(plus)  +' '+''.join(pro)  +' '+ ''.join(max)  +' '+''.join(year)
      new_name = re.sub('\(|\)',' ',new_name)
      new_name = re.sub('^ +','',new_name)
      new_name = re.sub(' +$','',new_name)
      new_name = re.sub(' +',' ',new_name)
      data_df['name'][i] = new_name.upper()
  except:
    continue

In [ ]:
list_idx = []
for i in range(0,len(data_df['name'])):
  try:
    ip = re.findall('(ipad)',data_df['name'][i].lower())
    ver = re.findall('( [0-9] |gen [0-9]| [0-9]$|m1)',data_df['name'][i].lower())
    year = re.findall('( [0-9][0-9][0-9][0-9] | [0-9][0-9][0-9][0-9]$)',data_df['name'][i].lower())
    pro = re.findall('(pro)',data_df['name'][i].lower())
    mini = re.findall('(mini)',data_df['name'][i].lower())
    air = re.findall('(air)',data_df['name'][i].lower())
    screen = re.findall("(11|12\.9|10\.5|9\.7|10\.2)",data_df['name'][i].lower())
    if ip != []:
      list_idx.append(i)
      new_name = ''.join(ip)+' '+''.join(pro) +' ' + ''.join(mini)+' ' +''.join(air) +' ' + ''.join(ver) +' ' + ''.join(screen) +' ' +''.join(year)
      new_name = re.sub('\(|\)|\"|\″',' ',new_name)
      new_name = re.sub('^ +','',new_name)
      new_name = re.sub(' +$','',new_name)
      new_name = re.sub(' +',' ',new_name)
      data_df['name'][i] = new_name.upper()
  except:
    continue

In [ ]:
for i in list_idx:
    ip = re.findall('(ipad)',data_df['name'][i].lower())
    ver = re.findall('( [0-9] |gen [0-9]| [0-9]$|m1)',data_df['name'][i].lower())
    year = re.findall('( [0-9][0-9][0-9][0-9] | [0-9][0-9][0-9][0-9]$)',data_df['name'][i].lower())
    pro = re.findall('(pro)',data_df['name'][i].lower())
    mini = re.findall('(mini)',data_df['name'][i].lower())
    air = re.findall('(air)',data_df['name'][i].lower())
    screen = re.findall("(11|12\.9|10\.5|9\.7|10\.2)",data_df['name'][i].lower())
    new_name = ''.join(ip)+' '+''.join(pro) +' ' + ''.join(mini)+' ' +''.join(air) +' ' + ''.join(ver) +' ' + ''.join(screen) +' ' +''.join(year)
    new_name = re.sub('\(|\)|\"|\″',' ',new_name)
    new_name = re.sub('^ +','',new_name)
    new_name = re.sub(' +$','',new_name)
    new_name = re.sub(' +',' ',new_name)
    data_df['name'][i] = new_name.upper()

In [ ]:
for i in list_idx:
  if(re.findall("IPAD AIR 3",data_df['name'][i])):
    data_df['name'][i] = 'IPAD AIR 3'
  if(re.findall("IPAD AIR 4",data_df['name'][i])):
    data_df['name'][i] = 'IPAD AIR 4'
  if(re.findall("IPAD GEN 5",data_df['name'][i])):
    data_df['name'][i] = 'IPAD GEN 5'
  if(re.findall("IPAD 2017",data_df['name'][i])):
    data_df['name'][i] = 'IPAD GEN 5'
  if(re.findall("IPAD 2018",data_df['name'][i])):
    data_df['name'][i] = 'IPAD GEN 6'
  if(re.findall("IPAD GEN 6",data_df['name'][i])):
    data_df['name'][i] = 'IPAD GEN 6'
  if(re.findall("IPAD GEN 7",data_df['name'][i])):
    data_df['name'][i] = 'IPAD GEN 7'
  if(re.findall("IPAD GEN 8",data_df['name'][i])):
    data_df['name'][i] = 'IPAD GEN 8'
  if(re.findall("IPAD MINI 5",data_df['name'][i])):
    data_df['name'][i] = 'IPAD MINI 5'
  if(re.findall("IPAD PRO M1 11",data_df['name'][i])):
    data_df['name'][i] = 'IPAD PRO M1 11'
  if(re.findall("IPAD PRO M1 12.9",data_df['name'][i])):
    data_df['name'][i] = 'IPAD PRO M1 12.9'

In [ ]:
# reformat all schema
for col in data_df:
  for i in range(0,len(data_df[col])):
    try:
      data_df[col][i] = re.sub('^ +','',data_df[col][i])
    except:
      continue

In [ ]:
def convertName(name):
  try:
    t_name = name.lower()
  except:
    return name
  if 'macbook' in t_name:
    #Pro or air
    f_name = 'MACBOOK'
    if 'air' in t_name:
      f_name += ' AIR'
    elif 'pro' in t_name:
      f_name += ' PRO'
    
    #M1 or not
    if 'm1' in t_name:
      f_name += ' M1'

    #inch
    if 'air' in t_name: 
      f_name += ' 13'
    elif '13' in t_name or '13"' in t_name or '13.3' in t_name:
      f_name += " 13"
    elif '15' in t_name:
      f_name += " 15"
    elif '16' in t_name:
      f_name += ' 16'
    elif '12' in t_name:
      f_name += ' 12'
    
    #touchbar
    if 'touchbar' in t_name:
      f_name += ' TOUCHBAR'
    
    #year
    x = re.findall(r'.*([1-3][0-9]{3})', t_name)
    if len(x) > 0:
      if int(x[0]) <=2021:
        f_name += (' ' + x[0])

    print(i,f_name, name)
    
    return f_name
  else:
    return name

In [ ]:
def convertWatchName(name):
  try:
    t_name = name.lower()
  except:
    return name
  if 'watch' in t_name:
    #Pro or air
    f_name = 'APPLE WATCH'

    #Series
    lst_serie = ['s1', 's2', 's3', 's4', 's5','s6', 'series 1', 'series 2', 'series 3', 'series 4', 'series 5', 'series 6' ]
    for j in range(0,len(lst_serie)):
      if lst_serie[j] in t_name:
        f_name += (' ' + lst_serie[j%6].upper())
        break
    if 'SE' in name:
      f_name += ' SE'

    #LTE or GPS
    if 'lte' in t_name:
      f_name += ' LTE'
    if 'gps' in t_name:
      f_name+= ' GPS'

    # Size
    size = name.find('mm')
    if size != -1 and size > 2:
      f_name += ' ' + t_name[size -3:size].strip('(- ')

    #year
    x = re.findall(r'.*([1-3][0-9]{3})', t_name)
    if len(x) > 0:
      if int(x[0]) <=2021:
        f_name += (' ' + x[0])

    print(f_name, name)
    
    return f_name
  else:
    return name

In [ ]:
data_df['name'] = data_df.apply(lambda row : convertName(row['name']), axis = 1)

4234 MACBOOK AIR 13 2018 Macbook Air 13″ 256GB Space Gray 2018
4234 MACBOOK AIR 13 2018 Macbook Air 13″ 128GB Gold 2018
4234 MACBOOK AIR 13 2018 Macbook Air 13″ 256GB Gold 2018
4234 MACBOOK AIR 13 2018 Macbook Air 13″ 256GB Silver 2018
4234 MACBOOK AIR 13 2018 Macbook Air 13″ 128GB Silver 2018
4234 MACBOOK AIR 13 2018 Macbook Air 13″ 128GB Space Gray 2018
4234 MACBOOK PRO 16 2019 Macbook Pro 16 inch 1TB 2019
4234 MACBOOK PRO 16 2019 Macbook Pro 16 inch 1TB 2019
4234 MACBOOK PRO 15 MacBook Pro 15 inch Touch Bar 512GB Silver (MR972)
4234 MACBOOK PRO 16 2019 Macbook Pro 16 inch 512GB 2019
4234 MACBOOK AIR 13 2020 Macbook Air 2020
4234 MACBOOK AIR 13 2020 Macbook Air 2020
4234 MACBOOK 12 2017 The New macbook (2017) 12″ – MNYJ2 – SILVER – 2017
4234 MACBOOK 12 2017 The New macbook (2017) 12″ – MNYL2 – Gold – 2017
4234 MACBOOK 2017 Macbook 2017
4234 MACBOOK 2017 Macbook 2017
4234 MACBOOK 12 2017 The New macbook (2017) 12″ – MNYN2 – Rose Gold – 2017
4234 MACBOOK AIR 13 2020 Macbook Air 2020
42

In [ ]:
data_df['name'] = data_df.apply(lambda row : convertWatchName(row['name']), axis = 1)

APPLE WATCH S4 40 Apple Watch Series 4 (40mm) Esim Mới 100% TBH
APPLE WATCH S4 40 Apple Watch Series 4 (40 mm) Cũ Like New 99%
APPLE WATCH S1 42 Apple Watch Series 1 (42mm) Viền Nhôm, Uy Tín, Chính Hãng, Giá Rẻ, Trả Góp 0%
APPLE WATCH S3 38 Apple Watch Series 3 (38mm) Mới 100% Trôi bảo hành
APPLE WATCH S4 44 Apple Watch Series 4 (44 mm) Cũ Like New 99%
APPLE WATCH S2 GPS Apple Watch Series 2 GPS (Thép)
APPLE WATCH S4 44 Apple Watch Series 4 (44 mm) Esim Mới 100% TBH
APPLE WATCH S3 LTE 38 Apple Watch Series 3 (38mm) LTE Bản Thép Cũ Like New 99%
APPLE WATCH S6 GPS 44 Apple Watch S6 GPS, Viền Nhôm SIze 44mm Mới 100% Fullbox Chính Hãng Apple
APPLE WATCH S6 GPS 40 Apple Watch S6 GPS, Viền Nhôm SIze 40mm Mới 100% Fullbox Chính Hãng Apple
APPLE WATCH SE GPS 40 Apple Watch SE GPS, Viền Nhôm SIze 40mm Mới 100% Fullbox Chính Hãng Apple
APPLE WATCH SE GPS 44 Apple Watch SE GPS, Viền Nhôm SIze 44mm Mới 100% Fullbox Chính Hãng Apple
APPLE WATCH S6 LTE 44 Apple Watch S6 LTE SIze 44mm  Mới 100% Fullb

In [ ]:
data_df.to_csv('new.csv')

# RECORD LINKAGE IPAD,IPHONE

```
# This is formatted as code
```


=> chạy xong ra stores_ip.csv là file chứa các thứ đã dc lọc trùng

In [ ]:
list_idx = []
for i in range(0,len(data_df['name'])):
    if isNaN(data_df['name'][i]) or isNaN(data_df['ram'][i]) or isNaN(data_df['rom'][i]):
        list_idx.append(i)
data_df.drop(list_idx,axis=0,inplace=True)
data_df.reset_index(drop=True, inplace=True)


In [ ]:
new_df = data_df
new_df.shape

(2643, 24)

Công thức xây dựng dựa trên các trường name, ram, rom , color, status

In [ ]:
# index some values by status block

indexer = recordlinkage.Index()
indexer.block('status')
candidate_links = indexer.index(new_df)

In [ ]:
# define the compare_cl vector and begin to compute them

compare_cl = recordlinkage.Compare()
compare_cl.string('name', 'name', method='damerau_levenshtein', threshold=1, label='name')
compare_cl.string('ram', 'ram', method='damerau_levenshtein',threshold=1, label='ram')
compare_cl.string('rom', 'rom', method='damerau_levenshtein',threshold=1, label='rom')
compare_cl.string('color', 'color', method='damerau_levenshtein',threshold=1, label='color')
compare_cl.string('status', 'status', method='damerau_levenshtein',threshold=1, label='status')

features = compare_cl.compute(candidate_links , new_df)

In [ ]:
# determine the points
features.sum(axis=1).value_counts().sort_index(ascending=False)

5.0     10864
4.0     72251
3.0    282735
2.0    712966
1.0    762635
dtype: int64

In [ ]:
# define the matches based on previous
matches = features[features.sum(axis=1) >= 5]
print(len(matches))
matches.head(10)

10864


name  ram  rom  color  status
9  5    1.0  1.0  1.0    1.0     1.0
16 6    1.0  1.0  1.0    1.0     1.0
18 6    1.0  1.0  1.0    1.0     1.0
   16   1.0  1.0  1.0    1.0     1.0
20 17   1.0  1.0  1.0    1.0     1.0
59 27   1.0  1.0  1.0    1.0     1.0
62 10   1.0  1.0  1.0    1.0     1.0
65 0    1.0  1.0  1.0    1.0     1.0
66 64   1.0  1.0  1.0    1.0     1.0
67 63   1.0  1.0  1.0    1.0     1.0

In [ ]:
# set name, duplication, and showing the shape after deduplication
matches.index = matches.index.set_names(['col1','col2'])


In [ ]:
duplicates = matches.index.get_level_values('col2')
prefix = matches.index.get_level_values('col1')
# datum = []
# for i in range(0,len(new_df.index)):
#   for j in range(0,len(duplicates)):
#     if i == duplicates[j]:
#       datum.append({i:prefix[j]})


In [ ]:
unique_b = new_df[~new_df.index.isin(duplicates)]
unique_b.shape

(671, 24)

In [ ]:
drop_cols = "link,pin,price,weight,resolution,screensize,screentech,gpu,size,rear_cam,front_cam,sim,tech,wifi,bluetooth,os,port".split(",")
norm_df = unique_b.drop(drop_cols,axis=1)
norm_df = norm_df.assign(place="")
norm_df = norm_df.assign(num_place="")


In [ ]:
for col in norm_df:
  print(col)

color
cpu
name
ram
rom
status
bluetoorh
place
num_place


In [ ]:
for i in norm_df.index:
  norm_df['place'][i] = str(i)
  for j in range(0,len(duplicates)):
    if i == duplicates[j]:
      norm_df['place'][i] = norm_df['place'][i] + ',' + str(prefix[j])
    if i == prefix[j]:
      norm_df['place'][i] = norm_df['place'][i] + ',' + str(duplicates[j])

In [ ]:
for i in norm_df.index:
  norm_df['num_place'][i] = str(len(norm_df['place'][i].split(',')))

In [ ]:
new_df = new_df.assign(unique_idx="")

In [ ]:
for i in norm_df.index:
  list_idx = []
  list_idx = norm_df['place'][i].split(',')
  for j in list_idx:
    new_df['unique_idx'][int(j)] = str(i)


In [ ]:
new_df.to_csv('ip_new.csv',index=False)

In [ ]:
norm_df.to_csv('stores_ip.csv',index_label='store_id')

In [ ]:
new_df.shape

(2643, 25)